# PID Control Testing for Maglev Pod

This notebook provides a PID control interface for the `LevPodEnv` simulation environment.

## Control Architecture
- **Height PID**: Controls average gap height → base PWM for all coils
- **Roll PID**: Corrects roll angle → differential left/right adjustment
- **Pitch PID**: Corrects pitch angle → differential front/back adjustment

The outputs combine: `coil_pwm = height_output ± roll_adjustment ± pitch_adjustment`

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from lev_pod_env import LevPodEnv, TARGET_GAP

# Set plot style for better aesthetics
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.facecolor'] = 'white'
plt.rcParams['axes.facecolor'] = 'white'
plt.rcParams['font.size'] = 11
plt.rcParams['axes.titlesize'] = 14
plt.rcParams['axes.labelsize'] = 12

print(f"Target Gap Height: {TARGET_GAP * 1000:.2f} mm")

---
## PID Controller Class

In [ ]:
class PIDController:
    """Simple PID controller with anti-windup."""
    
    def __init__(self, kp: float, ki: float, kd: float, 
                 output_min: float = -1.0, output_max: float = 1.0,
                 integral_limit: float = np.inf):
        self.kp = kp
        self.ki = ki
        self.kd = kd
        self.output_min = output_min
        self.output_max = output_max
        self.integral_limit = integral_limit if integral_limit else abs(output_max) * 2
        
        self.integral = 0.0
        self.prev_error = 0.0
        self.first_update = True
    
    def reset(self):
        """Reset controller state."""
        self.integral = 0.0
        self.prev_error = 0.0
        self.first_update = True
    
    def update(self, error: float, dt: float) -> float:
        """Compute PID output.
        
        Args:
            error: Current error (setpoint - measurement)
            dt: Time step in seconds
        
        Returns:
            Control output (clamped to output limits)
        """
        # Proportional term
        p_term = self.kp * error
        
        # Integral term with anti-windup
        self.integral += error * dt
        self.integral = np.clip(self.integral, -self.integral_limit, self.integral_limit)
        i_term = self.ki * self.integral
        
        # Derivative term (skip on first update to avoid spike)
        if self.first_update:
            d_term = 0.0
            self.first_update = False
        else:
            d_term = self.kd * (error - self.prev_error) / dt
        
        self.prev_error = error
        
        # Compute and clamp output
        output = p_term + i_term + d_term
        return np.clip(output, self.output_min, self.output_max)

---
## PID Gains Configuration

**Modify these values to tune the controller.**

- **Height PID**: Controls vertical position (gap height)
- **Roll PID**: Corrects side-to-side tilt
- **Pitch PID**: Corrects front-to-back tilt

In [ ]:
# ============================================================
#                    HEIGHT PID GAINS
# ============================================================
# Controls average gap height. Positive error = pod too low.
# Output: base PWM applied to all coils (positive = more lift)

HEIGHT_KP = 50.0    # Proportional gain
HEIGHT_KI = 5.0     # Integral gain  
HEIGHT_KD = 10.0    # Derivative gain

# ============================================================
#                     ROLL PID GAINS
# ============================================================
# Corrects roll angle (rotation about X-axis).
# Positive roll = right side lower → increase right coils
# Output: differential adjustment to left/right coils

ROLL_KP = 2.0       # Proportional gain
ROLL_KI = 0.5       # Integral gain
ROLL_KD = 0.5       # Derivative gain

# ============================================================
#                    PITCH PID GAINS  
# ============================================================
# Corrects pitch angle (rotation about Y-axis).
# Positive pitch = back lower → increase back coils
# Output: differential adjustment to front/back coils

PITCH_KP = 2.0      # Proportional gain
PITCH_KI = 0.5      # Integral gain
PITCH_KD = 0.5      # Derivative gain

print("PID Gains Configured:")
print(f"  Height: Kp={HEIGHT_KP}, Ki={HEIGHT_KI}, Kd={HEIGHT_KD}")
print(f"  Roll:   Kp={ROLL_KP}, Ki={ROLL_KI}, Kd={ROLL_KD}")
print(f"  Pitch:  Kp={PITCH_KP}, Ki={PITCH_KI}, Kd={PITCH_KD}")

---
## Simulation Runner

In [ ]:
def run_pid_simulation(
    initial_gap_mm: float = 14.0,
    max_steps: int = 2000,
    use_gui: bool = False,
    disturbance_step: int = None,
    disturbance_force: float = 0.0,
    disturbance_force_std: float = 0.0,
    verbose: bool = True
) -> dict:
    """
    Run a PID-controlled simulation of the maglev pod.

    Args:
        initial_gap_mm: Starting gap height in millimeters
        max_steps: Maximum simulation steps (each step is 1/240 second)
        use_gui: Whether to show PyBullet GUI visualization
        disturbance_step: Step at which to apply impulse disturbance (None = no impulse)
        disturbance_force: Impulse force in Newtons at disturbance_step (positive = upward)
        disturbance_force_std: Standard deviation of continuous stochastic noise (Newtons)
        verbose: Print progress updates

    Returns:
        Dictionary containing time series data for plotting
    """
    # Create environment with stochastic disturbance if specified
    env = LevPodEnv(
        use_gui=use_gui,
        initial_gap_mm=initial_gap_mm,
        max_steps=max_steps,
        disturbance_force_std=disturbance_force_std
    )
    dt = env.dt  # 1/240 seconds

    # Initialize PID controllers
    height_pid = PIDController(HEIGHT_KP, HEIGHT_KI, HEIGHT_KD, output_min=-1.0, output_max=1.0)
    roll_pid = PIDController(ROLL_KP, ROLL_KI, ROLL_KD, output_min=-0.5, output_max=0.5)
    pitch_pid = PIDController(PITCH_KP, PITCH_KI, PITCH_KD, output_min=-0.5, output_max=0.5)

    # Data storage
    data = {
        'time': [],
        'gap_front': [],
        'gap_back': [],
        'gap_avg': [],
        'roll_deg': [],
        'pitch_deg': [],
        'current_FL': [],
        'current_FR': [],
        'current_BL': [],
        'current_BR': [],
        'current_total': [],
        'pwm_FL': [],
        'pwm_FR': [],
        'pwm_BL': [],
        'pwm_BR': [],
    }

    # Reset environment
    obs, _ = env.reset()

    target_gap = TARGET_GAP  # meters

    if verbose:
        print(f"Starting simulation: initial_gap={initial_gap_mm}mm, target={target_gap*1000:.2f}mm")
        if disturbance_step:
            print(f"  Impulse disturbance: {disturbance_force}N at step {disturbance_step}")
        if disturbance_force_std > 0:
            print(f"  Stochastic noise: std={disturbance_force_std}N")

    for step in range(max_steps):
        t = step * dt

        # Extract gap heights from observation (first 4 values are normalized gaps)
        # These are sensor readings (may include noise in future)
        gaps_normalized = obs[:4]  # [center_right, center_left, front, back]
        gaps = gaps_normalized * env.gap_scale + TARGET_GAP

        gap_front = gaps[2]  # Front sensor
        gap_back = gaps[3]   # Back sensor
        gap_left = gaps[1]   # Center_Left sensor
        gap_right = gaps[0]  # Center_Right sensor

        gap_avg = (gap_front + gap_back + gap_left + gap_right) / 4

        # Calculate roll and pitch from sensor gaps (for PID control)
        y_distance = 0.1016  # Left to right distance (m)
        x_distance = 0.2518  # Front to back distance (m)

        roll_angle = np.arcsin(np.clip((gap_left - gap_right) / y_distance, -1, 1))
        pitch_angle = np.arcsin(np.clip((gap_back - gap_front) / x_distance, -1, 1))

        # Compute PID outputs
        height_error = target_gap - gap_avg  # Positive = too low, need more lift
        roll_error = -roll_angle  # Target is 0, negative feedback
        pitch_error = -pitch_angle  # Target is 0, negative feedback

        height_output = height_pid.update(height_error, dt)
        roll_output = roll_pid.update(roll_error, dt)
        pitch_output = pitch_pid.update(pitch_error, dt)

        # Combine outputs into 4 coil PWM commands
        # Coil layout: front_left (+x,+y), front_right (+x,-y), back_left (-x,+y), back_right (-x,-y)
        # Roll correction: positive roll_output → increase right side (FR, BR)
        # Pitch correction: positive pitch_output → increase back side (BL, BR)

        pwm_FL = np.clip(height_output - roll_output - pitch_output, -1, 1)
        pwm_FR = np.clip(height_output + roll_output - pitch_output, -1, 1)
        pwm_BL = np.clip(height_output - roll_output + pitch_output, -1, 1)
        pwm_BR = np.clip(height_output + roll_output + pitch_output, -1, 1)

        action = np.array([pwm_FL, pwm_FR, pwm_BL, pwm_BR], dtype=np.float32)

        # Apply impulse disturbance if specified (uses environment method)
        if disturbance_step and step == disturbance_step:
            env.apply_impulse(disturbance_force)
            if verbose:
                print(f"  Applied {disturbance_force}N impulse at step {step}")

        # Step environment (stochastic disturbance applied internally if enabled)
        obs, _, terminated, truncated, info = env.step(action)

        # Store ground truth data from environment info (not sensor observations)
        data['time'].append(t)
        data['gap_front'].append(info['gap_front_yoke'] * 1000)  # Convert to mm
        data['gap_back'].append(info['gap_back_yoke'] * 1000)
        data['gap_avg'].append(info['gap_avg'] * 1000)  # Use ground truth from info
        data['roll_deg'].append(np.degrees(info['roll']))
        data['pitch_deg'].append(np.degrees(info['pitch']))  # Use ground truth from info
        data['current_FL'].append(info['curr_front_L'])
        data['current_FR'].append(info['curr_front_R'])
        data['current_BL'].append(info['curr_back_L'])
        data['current_BR'].append(info['curr_back_R'])
        data['current_total'].append(
            abs(info['curr_front_L']) + abs(info['curr_front_R']) +
            abs(info['curr_back_L']) + abs(info['curr_back_R'])
        )
        data['pwm_FL'].append(pwm_FL)
        data['pwm_FR'].append(pwm_FR)
        data['pwm_BL'].append(pwm_BL)
        data['pwm_BR'].append(pwm_BR)

        if terminated or truncated:
            if verbose:
                print(f"  Simulation ended at step {step} (terminated={terminated})")
            break

    env.close()

    # Convert to numpy arrays
    for key in data:
        data[key] = np.array(data[key])

    if verbose:
        print(f"Simulation complete: {len(data['time'])} steps, {data['time'][-1]:.2f}s")
        print(f"  Final gap: {data['gap_avg'][-1]:.2f}mm (target: {target_gap*1000:.2f}mm)")
        print(f"  Final roll: {data['roll_deg'][-1]:.3f}°, pitch: {data['pitch_deg'][-1]:.3f}°")

    return data

---
## Plotting Functions

In [ ]:
def plot_results(data: dict, title_suffix: str = ""):
    """
    Create aesthetic plots of simulation results.
    
    Args:
        data: Dictionary from run_pid_simulation()
        title_suffix: Optional suffix for plot titles
    """
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    fig.suptitle(f'PID Control Performance{title_suffix}', fontsize=16, fontweight='bold', y=1.02)
    
    target_gap_mm = TARGET_GAP * 1000
    time = data['time']
    
    # Color palette
    colors = {
        'primary': '#2563eb',    # Blue
        'secondary': '#7c3aed',  # Purple
        'accent1': '#059669',    # Green
        'accent2': '#dc2626',    # Red
        'target': '#f59e0b',     # Orange
        'grid': '#e5e7eb'
    }
    
    # ========== Gap Height Plot ==========
    ax1 = axes[0, 0]
    ax1.plot(time, data['gap_avg'], color=colors['primary'], linewidth=2, label='Average Gap')
    ax1.plot(time, data['gap_front'], color=colors['secondary'], linewidth=1, alpha=0.6, label='Front Yoke')
    ax1.plot(time, data['gap_back'], color=colors['accent1'], linewidth=1, alpha=0.6, label='Back Yoke')
    ax1.axhline(y=target_gap_mm, color=colors['target'], linestyle='--', linewidth=2, label=f'Target ({target_gap_mm:.1f}mm)')
    
    ax1.set_xlabel('Time (s)')
    ax1.set_ylabel('Gap Height (mm)')
    ax1.set_title('Gap Height Over Time', fontweight='bold')
    ax1.legend(loc='best', framealpha=0.9)
    ax1.set_ylim([0, max(35, max(data['gap_avg']) * 1.1)])
    ax1.grid(True, alpha=0.3)
    
    # Add settling info
    final_error = abs(data['gap_avg'][-1] - target_gap_mm)
    ax1.text(0.98, 0.02, f'Final error: {final_error:.2f}mm', 
             transform=ax1.transAxes, ha='right', va='bottom',
             fontsize=10, bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    
    # ========== Roll Angle Plot ==========
    ax2 = axes[0, 1]
    ax2.plot(time, data['roll_deg'], color=colors['primary'], linewidth=2)
    ax2.axhline(y=0, color=colors['target'], linestyle='--', linewidth=1.5, alpha=0.7)
    ax2.fill_between(time, data['roll_deg'], 0, alpha=0.2, color=colors['primary'])
    
    ax2.set_xlabel('Time (s)')
    ax2.set_ylabel('Roll Angle (degrees)')
    ax2.set_title('Roll Angle Over Time', fontweight='bold')
    ax2.grid(True, alpha=0.3)
    
    max_roll = max(abs(data['roll_deg'].min()), abs(data['roll_deg'].max()))
    ax2.set_ylim([-max(1, max_roll * 1.2), max(1, max_roll * 1.2)])
    
    # ========== Pitch Angle Plot ==========
    ax3 = axes[1, 0]
    ax3.plot(time, data['pitch_deg'], color=colors['secondary'], linewidth=2)
    ax3.axhline(y=0, color=colors['target'], linestyle='--', linewidth=1.5, alpha=0.7)
    ax3.fill_between(time, data['pitch_deg'], 0, alpha=0.2, color=colors['secondary'])
    
    ax3.set_xlabel('Time (s)')
    ax3.set_ylabel('Pitch Angle (degrees)')
    ax3.set_title('Pitch Angle Over Time', fontweight='bold')
    ax3.grid(True, alpha=0.3)
    
    max_pitch = max(abs(data['pitch_deg'].min()), abs(data['pitch_deg'].max()))
    ax3.set_ylim([-max(1, max_pitch * 1.2), max(1, max_pitch * 1.2)])
    
    # ========== Current Draw Plot ==========
    ax4 = axes[1, 1]
    ax4.plot(time, data['current_FL'], linewidth=1.5, label='Front Left', alpha=0.8)
    ax4.plot(time, data['current_FR'], linewidth=1.5, label='Front Right', alpha=0.8)
    ax4.plot(time, data['current_BL'], linewidth=1.5, label='Back Left', alpha=0.8)
    ax4.plot(time, data['current_BR'], linewidth=1.5, label='Back Right', alpha=0.8)
    ax4.plot(time, data['current_total'], color='black', linewidth=2, label='Total', linestyle='--')
    
    ax4.set_xlabel('Time (s)')
    ax4.set_ylabel('Current (A)')
    ax4.set_title('Coil Current Draw Over Time', fontweight='bold')
    ax4.legend(loc='best', framealpha=0.9, ncol=2)
    ax4.grid(True, alpha=0.3)
    
    # Add average power info
    avg_total_current = np.mean(data['current_total'])
    ax4.text(0.98, 0.98, f'Avg total: {avg_total_current:.2f}A', 
             transform=ax4.transAxes, ha='right', va='top',
             fontsize=10, bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    
    plt.tight_layout()
    plt.show()
    
    return fig

In [ ]:
def plot_control_signals(data: dict):
    """
    Plot the PWM control signals sent to each coil.
    
    Args:
        data: Dictionary from run_pid_simulation()
    """
    fig, ax = plt.subplots(figsize=(12, 5))
    
    time = data['time']
    
    ax.plot(time, data['pwm_FL'], label='Front Left', linewidth=1.5, alpha=0.8)
    ax.plot(time, data['pwm_FR'], label='Front Right', linewidth=1.5, alpha=0.8)
    ax.plot(time, data['pwm_BL'], label='Back Left', linewidth=1.5, alpha=0.8)
    ax.plot(time, data['pwm_BR'], label='Back Right', linewidth=1.5, alpha=0.8)
    
    ax.axhline(y=0, color='gray', linestyle='-', linewidth=0.5)
    ax.axhline(y=1, color='red', linestyle='--', linewidth=1, alpha=0.5, label='Saturation')
    ax.axhline(y=-1, color='red', linestyle='--', linewidth=1, alpha=0.5)
    
    ax.set_xlabel('Time (s)')
    ax.set_ylabel('PWM Duty Cycle')
    ax.set_title('Control Signals (PWM)', fontsize=14, fontweight='bold')
    ax.legend(loc='best', framealpha=0.9)
    ax.set_ylim([-1.2, 1.2])
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    return fig

---
## Run Simulation

**Configure the simulation parameters below and run the cell.**

In [ ]:
# ============================================================
#               SIMULATION PARAMETERS
# ============================================================

INITIAL_GAP_MM = 14.0     # Starting gap height (mm). Target is ~16.49mm
MAX_STEPS = 2000          # Simulation steps (240 steps = 1 second)
USE_GUI = False           # Set to True to see PyBullet visualization

# Impulse disturbance (one-time force at a specific step)
DISTURBANCE_STEP = None   # Step at which to apply impulse (e.g., 500). None = disabled
DISTURBANCE_FORCE = 0.0   # Impulse force in Newtons (positive = upward push)

# Stochastic disturbance (continuous random noise each step)
DISTURBANCE_FORCE_STD = 0.0  # Std dev of random force noise (Newtons). 0 = disabled

# ============================================================

# Run simulation
results = run_pid_simulation(
    initial_gap_mm=INITIAL_GAP_MM,
    max_steps=MAX_STEPS,
    use_gui=USE_GUI,
    disturbance_step=DISTURBANCE_STEP,
    disturbance_force=DISTURBANCE_FORCE,
    disturbance_force_std=DISTURBANCE_FORCE_STD,
    verbose=True
)

---
## View Results

In [ ]:
# Plot main performance metrics
plot_results(results)

In [ ]:
# Plot control signals (optional)
plot_control_signals(results)

---
## Compare Multiple Initial Conditions

Run this cell to test the controller with different starting heights.

In [ ]:
def compare_initial_conditions(initial_gaps_mm: list, max_steps: int = 2000):
    """
    Compare PID performance across different initial conditions.
    
    Args:
        initial_gaps_mm: List of starting gap heights to test
        max_steps: Maximum steps per simulation
    """
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    fig.suptitle('PID Performance Comparison: Different Initial Conditions', 
                 fontsize=16, fontweight='bold', y=1.02)
    
    target_gap_mm = TARGET_GAP * 1000
    colors = plt.cm.viridis(np.linspace(0, 0.8, len(initial_gaps_mm)))
    
    all_results = []
    
    for i, gap in enumerate(initial_gaps_mm):
        print(f"Running simulation {i+1}/{len(initial_gaps_mm)}: initial_gap={gap}mm")
        data = run_pid_simulation(initial_gap_mm=gap, max_steps=max_steps, verbose=False)
        all_results.append((gap, data))
        
        label = f'{gap}mm'
        
        # Gap height
        axes[0, 0].plot(data['time'], data['gap_avg'], color=colors[i], 
                        linewidth=2, label=label)
        
        # Roll
        axes[0, 1].plot(data['time'], data['roll_deg'], color=colors[i], 
                        linewidth=2, label=label)
        
        # Pitch
        axes[1, 0].plot(data['time'], data['pitch_deg'], color=colors[i], 
                        linewidth=2, label=label)
        
        # Total current
        axes[1, 1].plot(data['time'], data['current_total'], color=colors[i], 
                        linewidth=2, label=label)
    
    # Add target line to gap plot
    axes[0, 0].axhline(y=target_gap_mm, color='red', linestyle='--', 
                       linewidth=2, label=f'Target ({target_gap_mm:.1f}mm)')
    
    # Configure axes
    axes[0, 0].set_xlabel('Time (s)')
    axes[0, 0].set_ylabel('Gap Height (mm)')
    axes[0, 0].set_title('Average Gap Height', fontweight='bold')
    axes[0, 0].legend(loc='best')
    axes[0, 0].grid(True, alpha=0.3)
    
    axes[0, 1].axhline(y=0, color='gray', linestyle='--', linewidth=1)
    axes[0, 1].set_xlabel('Time (s)')
    axes[0, 1].set_ylabel('Roll Angle (degrees)')
    axes[0, 1].set_title('Roll Angle', fontweight='bold')
    axes[0, 1].legend(loc='best')
    axes[0, 1].grid(True, alpha=0.3)
    
    axes[1, 0].axhline(y=0, color='gray', linestyle='--', linewidth=1)
    axes[1, 0].set_xlabel('Time (s)')
    axes[1, 0].set_ylabel('Pitch Angle (degrees)')
    axes[1, 0].set_title('Pitch Angle', fontweight='bold')
    axes[1, 0].legend(loc='best')
    axes[1, 0].grid(True, alpha=0.3)
    
    axes[1, 1].set_xlabel('Time (s)')
    axes[1, 1].set_ylabel('Total Current (A)')
    axes[1, 1].set_title('Total Current Draw', fontweight='bold')
    axes[1, 1].legend(loc='best')
    axes[1, 1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    return all_results

In [ ]:
# Compare different starting heights
# Uncomment and run to test multiple initial conditions

# comparison_results = compare_initial_conditions(
#     initial_gaps_mm=[10.0, 14.0, 18.0, 22.0],
#     max_steps=2000
# )

---
## Test Disturbance Rejection

Apply a sudden force disturbance and observe recovery.

In [ ]:
# Test disturbance rejection
# Uncomment and run to test disturbance response

# Example 1: Impulse disturbance (one-time force)
# disturbance_results = run_pid_simulation(
#     initial_gap_mm=16.5,         # Start near target
#     max_steps=3000,
#     use_gui=False,
#     disturbance_step=720,        # Apply at 3 seconds
#     disturbance_force=-20.0,     # 20N downward push
#     verbose=True
# )
# plot_results(disturbance_results, title_suffix=' (with 20N impulse at t=3s)')

# Example 2: Continuous stochastic noise
# noisy_results = run_pid_simulation(
#     initial_gap_mm=16.5,
#     max_steps=3000,
#     use_gui=False,
#     disturbance_force_std=2.0,   # 2N standard deviation continuous noise
#     verbose=True
# )
# plot_results(noisy_results, title_suffix=' (with 2N std continuous noise)')

---
## Summary Statistics

In [ ]:
def print_summary(data: dict):
    """Print summary statistics for a simulation run."""
    target_gap_mm = TARGET_GAP * 1000
    
    # Calculate settling time (within 2% of target)
    tolerance = 0.02 * target_gap_mm
    settled_idx = None
    for i in range(len(data['gap_avg'])):
        if abs(data['gap_avg'][i] - target_gap_mm) < tolerance:
            # Check if it stays settled
            if all(abs(data['gap_avg'][j] - target_gap_mm) < tolerance 
                   for j in range(i, min(i+100, len(data['gap_avg'])))):
                settled_idx = i
                break
    
    print("=" * 50)
    print("SIMULATION SUMMARY")
    print("=" * 50)
    print(f"Duration: {data['time'][-1]:.2f} seconds ({len(data['time'])} steps)")
    print(f"Target gap: {target_gap_mm:.2f} mm")
    print()
    print("Gap Height:")
    print(f"  Initial:  {data['gap_avg'][0]:.2f} mm")
    print(f"  Final:    {data['gap_avg'][-1]:.2f} mm")
    print(f"  Error:    {abs(data['gap_avg'][-1] - target_gap_mm):.3f} mm")
    print(f"  Max:      {max(data['gap_avg']):.2f} mm")
    print(f"  Min:      {min(data['gap_avg']):.2f} mm")
    if settled_idx:
        print(f"  Settling time (2%): {data['time'][settled_idx]:.3f} s")
    else:
        print(f"  Settling time: Not settled within tolerance")
    print()
    print("Orientation (final):")
    print(f"  Roll:  {data['roll_deg'][-1]:+.3f} degrees")
    print(f"  Pitch: {data['pitch_deg'][-1]:+.3f} degrees")
    print()
    print("Current Draw:")
    print(f"  Average total: {np.mean(data['current_total']):.2f} A")
    print(f"  Peak total:    {max(data['current_total']):.2f} A")
    print("=" * 50)

# Print summary for last simulation
print_summary(results)